In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

## Getting set up

In [38]:
comp = 'playground-series-s3e26'
path = setup_comp(comp, install='')

In [39]:
path

Path('playground-series-s3e26')

In [40]:
trn_path = path/'train.csv'

In [41]:
import pandas as pd
df = pd.read_csv(trn_path)

In [42]:
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.metrics import make_scorer, mean_absolute_error, classification_report, log_loss
from scipy.stats import loguniform

In [67]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]
X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2, stratify=y) # XXyy

In [68]:
ct = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include = np.number)),
    (OneHotEncoder(), make_column_selector(dtype_include=object)), 
    remainder = 'drop'
)

pipe = make_pipeline(ct, HistGradientBoostingClassifier())

pipe.fit(X_tr, y_tr)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f34b405be50>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f34b405a110>)])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [69]:
y_pred = pipe.predict(X_dev)
print(classification_report(y_dev,y_pred))

              precision    recall  f1-score   support

           C       0.85      0.91      0.88       993
          CL       0.60      0.11      0.18        55
           D       0.79      0.74      0.76       533

    accuracy                           0.83      1581
   macro avg       0.74      0.59      0.61      1581
weighted avg       0.82      0.83      0.82      1581



In [70]:
y_pred_proba = pipe.predict_proba(X_dev)
log_loss(y_dev, y_pred_proba)

0.4959340277517529

In [74]:
-np.mean(np.log(y_pred_proba[y_dev=='CL'][:,1]))

4.064452884639077

In [27]:
np.log(3)

1.0986122886681098

In [75]:
y_pred_proba[y_dev=='CL'] = np.array([1/3,1/3,1/3])

In [76]:
log_loss(y_dev, y_pred_proba)

0.39275804243967

## Submitting to Kaggle

In [77]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

,id,Status_C,Status_CL,Status_D
0,7905,0.628084,0.034788,0.337128
1,7906,0.628084,0.034788,0.337128
2,7907,0.628084,0.034788,0.337128
3,7908,0.628084,0.034788,0.337128
4,7909,0.628084,0.034788,0.337128
...,...,...,...,...
5266,13171,0.628084,0.034788,0.337128
5267,13172,0.628084,0.034788,0.337128
5268,13173,0.628084,0.034788,0.337128
5269,13174,0.628084,0.034788,0.337128


In [78]:
tst = pd.read_csv(path/'test.csv')
tst.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7905,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,7906,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,7907,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,7908,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,7909,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0


In [79]:
tst_pred = pipe.predict_proba(tst.iloc[:,1:])

In [80]:
tst_class = pipe.predict(tst.iloc[:,1:])

In [81]:
tst_pred[tst_class=='CL'] = np.array([1,1,1])

In [82]:
ss.iloc[:,1:] = tst_pred

In [83]:
ss

,id,Status_C,Status_CL,Status_D
0,7905,0.383739,0.002141,0.614120
1,7906,0.481318,0.072933,0.445749
2,7907,0.006800,0.000501,0.992699
3,7908,0.948806,0.000252,0.050942
4,7909,0.796934,0.047240,0.155826
...,...,...,...,...
5266,13171,0.972463,0.003441,0.024095
5267,13172,0.993935,0.000129,0.005936
5268,13173,0.963062,0.000399,0.036539
5269,13174,0.995731,0.000672,0.003598


In [84]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

id,Status_C,Status_CL,Status_D
7905,0.3837390964863589,0.002141234788570276,0.6141196687250708
7906,0.48131758554960785,0.07293310049323265,0.4457493139571595
7907,0.00680040754249639,0.0005005938294301725,0.9926989986280734
7908,0.9488057777473001,0.0002517897052014527,0.05094243254749845
7909,0.7969341597280866,0.04724009061615174,0.15582574965576176
7910,0.9983099860915601,7.087808395134713e-05,0.0016191358244887015
7911,0.9922384361714667,0.00019254276556593224,0.007569021062967292
7912,0.11034346225298095,0.008363961382498883,0.8812925763645202
7913,0.003009744679217623,6.403650134357082e-06,0.996983851670648


In [85]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'uniformise CL preds', comp)

100%|███████████████████████████████████████████████████████| 340k/340k [00:01<00:00, 321kB/s]


## Conclusion

## Addendum

In [26]:
if not iskaggle:
    push_notebook('xy', 'histgbr-minmax-transform',
                  title='Minmax transform and HistGBR model',
                  file='01-histgbr-minmax-transform.ipynb',
                  competition=comp, private=False, gpu=False)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/xiaochuanyang/minmax-transform-and-histgbr-model
